In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
import re

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
df = pd.read_csv("SteamApps.csv")
temp = pd.read_csv("steam_games.csv")
df = pd.concat([df, temp])
df

,Unnamed: 0,type,name,steam_appid,required_age,is_free,detailed_description,about_the_game,short_description,fullgame,...,controller_support,dlc,demos,achievements,reviews,recommendations,drm_notice,metacritic,alternate_appid,Unnamed: 0.1
0,0,demo,Pin Them Demo,1904630.0,0,True,NaN,NaN,NaN,"{'appid': '1764220', 'name': 'Pin Them'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,game,Al-Qadim: The Genie's Curse,1904640.0,0,False,"As an outcast, betrothed to the caliph's daugh...","As an outcast, betrothed to the caliph's daugh...",Experience the mysterious Al-Qadim game world ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,game,Dungeons & Dragons - Stronghold: Kingdom Simul...,1904650.0,0,False,Build and command your own kingdom in the Dung...,Build and command your own kingdom in the Dung...,Run your own kingdom in the legendary Dungeons...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,game,Chapel 3-D: The Ascent,1904680.0,0,False,"<h1>🔥 WISHLIST TO SURVIVE 🔥</h1><p><img src=""h...","<a href=""https://steamcommunity.com/linkfilter...","🔥 Chapel 3-D: The Ascent is a break-neck, viol...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,game,VTuber Gallery : Anime Pose,1904690.0,0,True,<strong>VTuber Gallery</strong> is <strong>#1 ...,<strong>VTuber Gallery</strong> is <strong>#1 ...,VTuber Gallery is #1 anime pose app that allow...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20554,270,video,The Chair,966800.0,13,False,<h1>Steam Video</h1><p>This content is only av...,Richard Sullivan has spent ten years on death ...,Richard Sullivan has spent ten years on death ...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270.0
20555,271,video,Head,966810.0,13,False,<h1>Steam Video</h1><p>This content is only av...,A group of 20-something puppets go on a weeken...,A group of 20-something puppets go on a weeken...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,271.0
20556,272,video,The Barn,966820.0,13,False,<h1>Steam Video</h1><p>This content is only av...,It is Halloween 1989. Best friends Sam and Jos...,Halloween 1989. Sam and Josh are enjoying what...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,272.0
20557,273,video,Be My Cat: A Film for Anne,966830.0,13,False,<h1>Steam Video</h1><p>This content is only av...,Obsessed with convincing Hollywood actress Ann...,Obsessed with convincing Hollywood actress Ann...,NaN,...,full,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,273.0


In [14]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.drop(columns=["header_image", "capsule_image", "capsule_imagev5", "website",
                 "screenshots", "background", "background_raw", "content_descriptors",
                 "support_info", "short_description", "about_the_game", "short_description",
                 "detailed_description", "legal_notice", "pc_requirements", "mac_requirements",
                 "linux_requirements", "package_groups", "movies"], inplace=True)
df.reset_index(drop=True, inplace=True)
df.head(3)

,type,name,steam_appid,required_age,is_free,detailed_description,fullgame,developers,publishers,package_groups,...,ext_user_account_notice,controller_support,dlc,demos,achievements,reviews,recommendations,drm_notice,metacritic,alternate_appid
0,demo,Pin Them Demo,1904630.0,0,True,NaN,"{'appid': '1764220', 'name': 'Pin Them'}",['PRODUKTIVKELLER Studios'],['PRODUKTIVKELLER Studios'],[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,game,Al-Qadim: The Genie's Curse,1904640.0,0,False,"As an outcast, betrothed to the caliph's daugh...",NaN,['Cyberlore Studios'],['SNEG'],"[{'name': 'default', 'title': ""Buy Al-Qadim: T...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,game,Dungeons & Dragons - Stronghold: Kingdom Simul...,1904650.0,0,False,Build and command your own kingdom in the Dung...,NaN,['Stormfront Studios'],['SNEG'],"[{'name': 'default', 'title': 'Buy Dungeons & ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# count = 0

# def html_string_to_json(x):
#     try:
#         if pd.isna(x):
#             return {}
#         else:
#             return json.loads(x.replace("\"", "").replace("\'", "\""))
#     except:
#         raise Exception(x)

# def get_minimum_requirement(x):
#     global count
#     if count % 10000 == 0:
#         print(f"Count at {count}")
#     count += 1
#     try:
#         temp = json.loads(x.replace("\"", "").replace("\'", "\""))
#         bs_model = BeautifulSoup(temp["minimum"])
#         return bs_model.text.replace("Minimum:", "")
#     except:
#         return None

# def get_recommend_requirement(x):
#     global count
#     if count % 10000 == 0:
#         print(f"Count at {count}")
#     count += 1
#     try:
#         temp = json.loads(x.replace("\"", "").replace("\'", "\""))
#         bs_model = BeautifulSoup(temp["recommended"])
#         return bs_model.text.replace("Recommended:", "")
#     except:
#         return None

In [16]:
# count = 0
# df["min_require"] = df["pc_requirements"].apply(get_minimum_requirement)
# print()
# count = 0
# df["rec_require"] = df["pc_requirements"].apply(get_recommend_requirement)
# df.drop(columns=["pc_requirements"], inplace=True)

In [28]:
def true_false_string_to_json(x):
    if pd.isna(x):
        return {}
    try:
        return json.loads(x.replace("\'", "\"").replace("\"True\"", "true").replace("\"False\"", "false"))
    except:
        return json.loads(x.replace("\'", "\"").replace("True", "true").replace("False", "false"))

def simple_string_to_json(x):
    if pd.isna(x):
        return {}
    else:
        return json.loads(x.replace("\'", "\""))

In [18]:
df["platforms"] = df["platforms"].apply(true_false_string_to_json)

In [19]:
df["platform_windows"] = df["platforms"].apply(lambda x: x.get("windows"))
df["platform_mac"] = df["platforms"].apply(lambda x: x.get("mac"))
df["platform_linux"] = df["platforms"].apply(lambda x: x.get("linux"))
df.drop(columns=["platforms"], inplace=True)

In [20]:
def get_average_age_rating(x, method=0):

    avg_val = 0
    val_list = []
    
    try:
        if not isinstance(x, str):
            return x
            
        json_x = json.loads(x.replace("\'", "\""))
        if "dejus" in json_x:
            keyword = "rating" if "rating" in json_x["dejus"] else "required_age"
            try:
                val_list.append(int(json_x["dejus"][keyword].replace("a", "").replace("l", "0")))
            except:
                pass
    
        if "cero" in json_x:
            keyword = "rating" if "rating" in json_x["cero"] else "required_age"
            try:
                match json_x["cero"][keyword]:
                    case "a": val_list.append(0)
                    case "b": val_list.append(12)
                    case "c": val_list.append(15)
                    case "d": val_list.append(17)
                    case "z": val_list.append(18)
            except:
                pass
    
        if "esrb" in json_x:
            keyword = "rating" if "rating" in json_x["esrb"] else "required_age"
            try:
                match json_x["esrb"][keyword]:
                    case "c": val_list.append(3)
                    case "e": val_list.append(7)
                    case "e10": val_list.append(12)
                    case "t": val_list.append(16)
                    case "m": val_list.append(17)
                    case "a": val_list.append(18)
            except:
                pass
    
        if "pegi" in json_x:
            keyword = "rating" if "rating" in json_x["pegi"] else "required_age"
            try:
                val_list.append(int(json_x["pegi"][keyword]))
            except:
                pass
    
        if "usk" in json_x:
            keyword = "rating" if "rating" in json_x["usk"] else "required_age"
            try:
                val_list.append(int(json_x["usk"][keyword]))
            except:
                pass
    
        if len(val_list) == 0:
            return 0
        else:
            for val in val_list:
                avg_val += val
            avg_val /= len(val_list)
            return avg_val

    except Exception as e:
        match method:
            case 0:
                return get_average_age_rating(x.replace("You\'re", "U re")  \
                                               .replace("\'CI\'", "") \
                                               .replace(" d\'", " d")  \
                                               .replace("t\'s", "t is"), 1)
            case 1:
                return None

In [21]:
df["avg_age"] = df["ratings"].apply(get_average_age_rating)
df.drop(columns=["ratings"], inplace=True)

In [24]:
def convert_to_datetime(x):
    try:
        return pd.to_datetime(x)
    except:
        return None

In [23]:
df["release_date"] = df["release_date"].apply(true_false_string_to_json)

In [25]:
df["coming_date"] = df["release_date"].apply(lambda x: convert_to_datetime(x["date"]))
df["coming_soon"] = df["release_date"].apply(lambda x: x["coming_soon"])
df.drop(columns=["release_date"], inplace=True)

In [26]:
df["package_number"] = df["package_groups"].apply(lambda x: len(x))
df.drop(columns=["package_number"], inplace=True)

In [29]:
df["price_overview"] = df["price_overview"].apply(simple_string_to_json)

In [30]:
df["currency"] = df["price_overview"].apply(lambda x: x.get("currency", None))
df["initial_price"] = df["price_overview"].apply(lambda x: x.get("initial", None))
df["final_price"] = df["price_overview"].apply(lambda x: x.get("final", None))
df["discount_percent"] = df["price_overview"].apply(lambda x: x.get("discount_percent", None))
df.drop(columns=["price_overview"], inplace=True)

In [38]:
df.loc[10, :]

type                                                                    game
name                                                                   REBEL
steam_appid                                                        1904800.0
required_age                                                               0
is_free                                                                False
fullgame                                                                 NaN
developers                                                        ['LTZinc']
publishers                                                        ['LTZinc']
categories                 [{'id': 2, 'description': 'Single-player'}, {'...
supported_languages                                                  English
packages                                                            [685984]
genres                     [{'id': '1', 'description': 'Action'}, {'id': ...
ext_user_account_notice                                                  NaN

In [21]:
df.loc[10, :]

Unnamed: 0                                                                10
type                                                                    game
name                                                                   REBEL
steam_appid                                                        1904800.0
required_age                                                               0
is_free                                                                False
detailed_description       REBEL - is a 3D shooter with surviving element...
about_the_game             REBEL - is a 3D shooter with surviving element...
short_description          REBEL - is a 3D shooter with surviving element...
fullgame                                                                 NaN
header_image               https://cdn.akamai.steamstatic.com/steam/apps/...
capsule_image              https://cdn.akamai.steamstatic.com/steam/apps/...
capsule_imagev5            https://cdn.akamai.steamstatic.com/steam/apps/...

In [47]:
df.to_csv("partly_cleaned.csv")